# 9 Paper Figures

The goal of the following notebook is to generate the figures included in our manuscript
___

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from glob import glob
import json
from plot_utils import plot_errorbars

In [ ]:
OUTPUT_PATH = '../results/imgs'

SAVE_FIGS = True

# Main Experiment

#### Parameters

In [ ]:
path_results_total_co2_osm = "../results/dict_total_co2_osm.json"
path_results_total_co2_tomtom = "../results/dict_total_co2_tomtom.json"

path_results_alpha_osm = "../results/dict_alpha_co2_osm.json"
path_results_alpha_tomtom = "../results/dict_alpha_co2_tomtom.json"

path_results_gini_osm = "../results/dict_gini_co2_osm.json"
path_results_gini_tomtom = "../results/dict_gini_co2_tomtom.json"

In [ ]:
## Total CO2 and alpha ##

map__navig__scenario__overallCO2 = {}
map__navig__scenario__alpha = {}
map__navig__scenario__gini = {}


## Total CO2

# OSM
f = open(path_results_total_co2_osm)
map__scenario__overallco2 = json.load(f)
map__navig__scenario__overallCO2['OSM'] = {int(key.split('_')[0]): value for key,value in map__scenario__overallco2.items()}

# TomTom
f = open(path_results_total_co2_tomtom)
map__scenario__overallco2 = json.load(f)
map__navig__scenario__overallCO2['TT'] = {int(key.split('_')[0]): value for key,value in map__scenario__overallco2.items()}


## Alphas
# OSM
f = open(path_results_alpha_osm)
map__scenario__alpha = json.load(f)
map__navig__scenario__alpha['OSM'] = {int(key.split('_')[0]): value for key,value in map__scenario__alpha.items()}

# TomTom
f = open(path_results_alpha_tomtom)
map__scenario__alpha = json.load(f)
map__navig__scenario__alpha['TT'] = {int(key.split('_')[0]): value for key,value in map__scenario__alpha.items()}


## Ginis

# OSM
f = open(path_results_gini_osm)
map__scenario__gini = json.load(f)
map__navig__scenario__gini['OSM'] = {int(key.split('_')[0]): value for key,value in map__scenario__gini.items()}

# TomTom
f = open(path_results_gini_tomtom)
map__scenario__gini = json.load(f)
map__navig__scenario__gini['TT'] = {int(key.split('_')[0]): value for key,value in map__scenario__gini.items()}



### Gini index
Code for plotting Figure 4a.

In [ ]:
from plot_utils import plot_errorbars

# modifying the keys [10,...,100] to [1,...,10]
map__navig__scenario__overallCO2__ = {}
for navig in map__navig__scenario__gini.keys():
    map__navig__scenario__overallCO2__[navig] = {int(scenario/10): map__navig__scenario__gini[navig][scenario] for scenario in map__navig__scenario__gini[navig].keys()}
#

fig, ax = plot_errorbars(map__navig__scenario__gini, 
                         plot_mean=True, 
                         x_label=r"$R$-routed (%)", #r"$i$ of $\overline{D}^{(R)}_{i}$"
                         y_label=r"Gini", 
                         figsize=(6,6))
#ax.set_ylim(0.855,0.88)

if SAVE_FIGS:
    plt.savefig(OUTPUT_PATH + "plot_errorbars_gini__V2.png", dpi=300)
    plt.savefig(OUTPUT_PATH + "plot_errorbars_gini__V2.pdf")

### Total CO$_2$
Code for plotting Figure 4b.

In [ ]:
from plot_utils import plot_errorbars

# modifying the keys [10,...,100] to [1,...,10]
map__navig__scenario__overallCO2__ = {}
for navig in map__navig__scenario__overallCO2.keys():
    map__navig__scenario__overallCO2__[navig] = {int(scenario/10): map__navig__scenario__overallCO2[navig][scenario] for scenario in map__navig__scenario__overallCO2[navig].keys()}
#

fig, ax = plot_errorbars(map__navig__scenario__overallCO2, 
                         plot_mean=True, 
                         x_label=r"$R$-routed (%)", #r"$i$ of $\overline{D}^{(R)}_{i}$"
                         y_label=r"Total CO$_2$ (mg)", 
                         figsize=(6,6))

if SAVE_FIGS:
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_AVG_total_co2__V2.png', dpi=300)
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_AVG_total_co2__V2.pdf')

### Truncated power-law's exponent $\alpha$
Code for plotting Figure A.1.

In [ ]:
from plot_utils import plot_errorbars

# modifying the keys [10,...,100] to [1,...,10]
map__navig__scenario__alpha__ = {}
for navig in map__navig__scenario__alpha.keys():
    map__navig__scenario__alpha__[navig] = {int(scenario/10): map__navig__scenario__alpha[navig][scenario] for scenario in map__navig__scenario__alpha[navig].keys()}
#

fig, ax = plot_errorbars(map__navig__scenario__alpha, 
                         plot_mean=True, 
                         x_label=r"$R$-routed (%)", #r"$i$ of $\overline{D}^{(R)}_{i}$"
                         y_label=r"Exponent $\mathbf{\alpha}$ of tr. power-law", 
                         figsize=(6,6))

if SAVE_FIGS:
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_AVG_alpha__V2.png', dpi=300)
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_AVG_alpha__V2.pdf')

# Experiment $w$ factor


In [ ]:
ws_to_plot = [1,15]
navigators = ["osm", "tomtom"]

#### parameters

In [ ]:
path_results_total_co2_osm_w = "../results/dict_total_co2_osm_w.json"
path_results_total_co2_tomtom_w = "../results/dict_total_co2_tomtom_w.json"

path_results_alpha_osm_w = "../results/dict_alpha_osm_w.json"
path_results_alpha_tomtom_w = "../results/results_alpha_tomtom_w.json"

path_results_gini_osm_w = "../results/dict_gini_osm_w.json"
path_results_gini_tomtom_w = "../results/results_gini_tomtom_w.json"

In [ ]:
dict_ws = {}

for navigator in navigators:
    
    dict_ws[navigator] = {}
    dict_ws[navigator]["total_co2"] = {}
    dict_ws[navigator]["alpha"] = {}
    dict_ws[navigator]["gini"] = {}
    dict_ws[navigator]["traveltime"] = {}

    for w in ws_to_plot:
        
        f = open(f"../results/dict_total_co2_{navigator}_w{w}.json")
        d = json.load(f)
        d = {int(k.split("_")[0]): d[k] for k in d}
        dict_ws[navigator]["total_co2"][str(w)] = d
         
        f = open(f"../results/dict_alpha_co2_{navigator}_w{w}.json")
        d = json.load(f)
        d = {int(k.split("_")[0]): d[k] for k in d}
        dict_ws[navigator]["alpha"][str(w)] = d
        
        f = open(f"../results/dict_gini_co2_{navigator}_w{w}.json")
        d = json.load(f)
        d = {int(k.split("_")[0]): d[k] for k in d}
        dict_ws[navigator]["gini"][str(w)] = d
        
                
        f = open(f"../results/dict_tt_{navigator}_w{w}.json")
        d = json.load(f)
        d = {int(k.split("_")[0]): d[k] for k in d}
        dict_ws[navigator]["traveltime"][str(w)] = d
    
 

### Gini index
Figures 6a and 6d.

#### 6a (OSM)

In [ ]:
navigator = "osm"

fig, ax = plot_errorbars(dict_ws[navigator]["gini"], 
                         plot_mean=True, 
                         x_label=r"%s-routed" %navigator + " (%)",
                         y_label="Gini", 
                         cmap_name='copper_r',
                         legend_loc = 'best', legend_title=r'Values of $\mathcal{w}$', legend_cols=2,
                         figsize=(6,6))

if SAVE_FIGS:
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_gini_%s.png' %navigator, dpi=300)
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_gini_%s.pdf' %navigator)

#### 6d (TomTom)

In [ ]:
navigator = "tomtom"

fig, ax = plot_errorbars(dict_ws[navigator]["gini"], 
                         plot_mean=True, 
                         x_label=r"%s-routed" %navigator + " (%)",
                         y_label="Gini", 
                         cmap_name='copper_r',
                         legend_loc = 'best', legend_title=r'Values of $\mathcal{w}$', legend_cols=2,
                         figsize=(6,6))

if SAVE_FIGS:
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_gini_%s.png' %navigator, dpi=300)
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_gini_%s.pdf' %navigator)

### Total CO2
Figures 6b and 6e.

#### 6b (OSM)

In [ ]:
navigator = "osm"
    
fig, ax = plot_errorbars(dict_ws[navigator]["total_co2"], 
                         plot_mean=True, 
                         x_label=r"OSM-routed" + " (%)", #r"$i$ of $\overline{D}^{(%s)}_{i}$" %navigator
                         y_label=r"Total CO$_2$ (mg)", 
                         cmap_name='copper_r',
                         legend_loc = 'best', legend_title=r'Values of $\mathcal{w}$', legend_cols=2,
                         figsize=(6,6))

if SAVE_FIGS:
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_totalco2_osm.png', dpi=300)
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_totalco2_osm.pdf')

#### 6e (TomTom)

In [ ]:
navigator = "osm"

fig, ax = plot_errorbars(dict_ws[navigator]["total_co2"], 
                         plot_mean=True, 
                         x_label=r"TT-routed" + " (%)", #r"$i$ of $\overline{D}^{(%s)}_{i}$" %navigator
                         y_label=r"Total CO$_2$ (mg)", 
                         cmap_name='copper_r',
                         legend_loc = 'best', legend_title=r'Values of $\mathcal{w}$', legend_cols=2,
                         figsize=(6,6))

if SAVE_FIGS:
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_totalco2_tomtom.png', dpi=300)
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_totalco2_tomtom.pdf')

### Travel Time
Figures 6c and 6f.

#### 6c (OSM)

In [ ]:
navigator = "osm"

fig, ax = plot_errorbars(dict_ws[navigator]["traveltime"], 
                         plot_mean=True, 
                         x_label=r"OSM-routed" + " (%)", #r"$i$ of $\overline{D}^{(%s)}_{i}$" %navigator
                         y_label=r"Total CO$_2$ (mg)", 
                         cmap_name='copper_r',
                         legend_loc = 'best', legend_title=r'Values of $\mathcal{w}$', legend_cols=2,
                         figsize=(6,6))

if SAVE_FIGS:
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_traveltime_osm.png', dpi=300)
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_traveltime_osm.pdf')

#### 6f (TomTom)

In [ ]:
navigator = "tomtom"

fig, ax = plot_errorbars(dict_ws[navigator]["traveltime"], 
                         plot_mean=True, 
                         x_label=r"TT-routed" + " (%)", #r"$i$ of $\overline{D}^{(%s)}_{i}$" %navigator
                         y_label=r"Total CO$_2$ (mg)", 
                         cmap_name='copper_r',
                         legend_loc = 'best', legend_title=r'Values of $\mathcal{w}$', legend_cols=2,
                         figsize=(6,6))

if SAVE_FIGS:
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_traveltime_tomtom.png', dpi=300)
    plt.savefig(OUTPUT_PATH + '/plot_errorbars_w_analysis_traveltime_tomtom.pdf')